<a href="https://colab.research.google.com/github/RudyMartin/dsai-2024/blob/main/pt_train_rps_cnn_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PYTORCH** Rock Paper Scissors with CNN Models
Assumes running T4 GPU Backend on Google Colab

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import os
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import datetime

# Start time
start_time = datetime.datetime.now()

# 1. Mount Google Drive to access the dataset
drive.mount('/content/gdrive')

# 2. Verify and list directories
root_dir = '/content/gdrive/My Drive/'
print(os.listdir(root_dir))

rps_dir = os.path.join(root_dir, 'rps')
if os.path.exists(rps_dir):
    print(f"'rps' directory contents: {os.listdir(rps_dir)}")
else:
    raise FileNotFoundError(f"Directory {rps_dir} does not exist.")

train_dir = os.path.join(rps_dir, 'train')
test_dir = os.path.join(rps_dir, 'test')
model_dir = os.path.join(root_dir, 'model')
print(f"rps directory contents: {os.listdir(rps_dir)}")

# 3. Data augmentation and normalization for training
train_transforms = transforms.Compose([
    transforms.Resize((160, 120)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(40),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=None, shear=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# 4. Just normalization for testing
test_transforms = transforms.Compose([
    transforms.Resize((160, 120)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# 5. Load datasets
train_dataset = ImageFolder(os.path.join(rps_dir, 'train'), transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

test_dataset = ImageFolder(os.path.join(rps_dir, 'test'), transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 6. Define the CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 20 * 15, 256)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 128)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(128, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

model = CNNModel().to(device)

# 7. Compile the model
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

# 8. Define a learning rate schedule (if needed)
def scheduler(epoch):
    if epoch < 10:
        return 1.0
    else:
        return np.exp(-0.1)

lr_scheduler = LambdaLR(optimizer, lr_lambda=scheduler)

# 9. Train the model
def train_model(epochs):
    model.train()
    for epoch in range(epochs):
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
        lr_scheduler.step()

train_model(100)

# 10. Save the trained model
model_path = os.path.join(rps_dir, 'pt_model.pth')
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}.")

# 11. Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test accuracy: {accuracy:.2f}%')

# 12. Accuracy per class
Y_pred = []
actual_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        Y_pred.extend(predicted.cpu().numpy())
        actual_labels.extend(labels.cpu().numpy())

print('Confusion Matrix')
print(confusion_matrix(actual_labels, Y_pred))
print('Classification Report')
target_names = list(test_loader.dataset.classes)
# Assuming actual_labels and Y_pred are your actual and predicted labels respectively
# and target_names are the names of the classes
print(classification_report(actual_labels, Y_pred, target_names=target_names, zero_division=0))


# End time
end_time = datetime.datetime.now()
# Calculate duration
duration = end_time - start_time
# Print execution time
print(f"Execution time: {str(duration)}")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
['DSC02677.jpeg', 'Colab Notebooks', 'dscamp_2023', 'dscamp_2022', 'proportions.csv', 'proportions.gsheet', 'activeloop_2023', 'Untitled spreadsheet (1).gsheet', 'Untitled spreadsheet.gsheet', 'YouTube', 'Copy of My Presentation.gslides', 'Introduction .gslides', 'Copy of Introduction .gslides', 'dscamp', 'dscamp_2024_nano', 'ds_camp_2024_trans_hf', 'rps_test', 'rps', 'rps_pics', 'papers']
'rps' directory contents: ['test', 'train', 'models', 'modelsbaseline_1.keras', 'sequential_acc_graph.png', 'sequential_loss_graph.png', 'df_metrics_20240610.csv', 'model.h5', 'model.keras', 'model.pth', 'pt_model.pth']
rps directory contents: ['test', 'train', 'models', 'modelsbaseline_1.keras', 'sequential_acc_graph.png', 'sequential_loss_graph.png', 'df_metrics_20240610.csv', 'model.h5', 'model.keras', 'model.pth', 'pt_model.pth']
Using device: cuda
Model saved to /con